# Data Science Capstone Project

### Import Required libraries

In [4]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
import requests

### Acquiring data from wikipedia with BeautifulSoup 

In [8]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = bs(source,'html.parser')

### Inputing data into data frame

In [26]:
postal_code=[]
brough=[]
neigh=[]
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        postal_code.append(cells[0].text.rstrip('\n'))
        brough.append(cells[1].text.rstrip('\n'))
        neigh.append(cells[2].text.rstrip('\n'))

In [27]:
col= ['Postal Code','Borough','Neighborhood']
df= pd.DataFrame(columns=col)
df['Postal Code']=postal_code
df['Borough']=brough
df['Neighborhood']=neigh
df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
